In [1]:
# import required modules
import os
import time
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, balanced_accuracy_score # balanced_accuracy_score with adjusted=True is Informedness
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing

df = pd.read_csv('Datasets for Assignment 3/census-income.csv')






When to One-Hot Encode vs Label Encode?

```To prevent biases from being introduced, One-Hot Encoding is preferable for nominal data (where there is no inherent order among categories). Label encoding, however, might be more appropriate for ordinal data (where categories naturally have an order)```

So we should one-hot encode columns like class of worker, state of residence, etc. After reviewing the colums I decided to one-hot encode all the following columns:

'ACLSWKR', 'ADTIND', 'ADTOCC', 'AMARITL', 'AMJIND', 'AMJOCC', 'ARACE', 'AREORGN', 'ASEX', 'AUNMEM', 'AUNTYPE', 'AWKSTAT', 'FILESTAT', 'GRINREG', 'GRINST', 'HHDFMX', 'HHDREL', 'MIGMTR1', 'MIGMTR3', 'MIGMTR4', 'PARENT', 'PEFNTVTY', 'PEMNTVTY', 'PENATVTY', 'PRCITSHP', 'SEOTR'

When I tried to run this I got:
Expected 2D array, got 1D array instead:

**https://www.geeksforgeeks.org/ml-one-hot-encoding-of-datasets-in-python/**



In [4]:
from sklearn.preprocessing import OneHotEncoder
OneHotEncode = OneHotEncoder()

columns_to_one_hot_encode = ['ACLSWKR', 'ADTIND', 'ADTOCC', 'AMARITL', 'AMJIND', 'AMJOCC', 'ARACE', 'AREORGN', 'ASEX', 'AUNMEM', 'AUNTYPE', 'AWKSTAT', 'FILESTAT', 'GRINREG', 'GRINST', 'HHDFMX', 'HHDREL', 'MIGMTR1', 'MIGMTR3', 'MIGMTR4', 'PARENT', 'PEFNTVTY', 'PEMNTVTY', 'PENATVTY', 'PRCITSHP', 'SEOTR']

for item in columns_to_one_hot_encode:
    df[item] = df[item].astype('category') # Must convert the strings to category numbers for One Hot to work
    df[item + '_new'] = df[item].cat.codes
    print(f" Column: {item}")
    print(df[item + '_new'])

OneHot_df = pd.DataFrame(OneHotEncode.fit_transform(df[columns_to_one_hot_encode]).toarray())

PostOneHot_df = df.join(OneHot_df) # Appends the OneHot_df to the original dataframe to create a new one
PostOneHot_df[:-100] # Check results from the One Hot Encoding


 Column: ACLSWKR
0         3
1         6
2         3
3         3
4         3
         ..
199518    3
199519    5
199520    3
199521    3
199522    4
Name: ACLSWKR_new, Length: 199523, dtype: int8
 Column: ADTIND
0          0
1          4
2          0
3          0
4          0
          ..
199518     0
199519    37
199520     0
199521     0
199522    42
Name: ADTIND_new, Length: 199523, dtype: int8
 Column: ADTOCC
0          0
1         34
2          0
3          0
4          0
          ..
199518     0
199519     2
199520     0
199521     0
199522    30
Name: ADTOCC_new, Length: 199523, dtype: int8
 Column: AMARITL
0         6
1         0
2         4
3         4
4         4
         ..
199518    2
199519    2
199520    2
199521    4
199522    4
Name: AMARITL_new, Length: 199523, dtype: int8
 Column: AMJIND
0         14
1          4
2         14
3         14
4         14
          ..
199518    14
199519     2
199520    14
199521    14
199522    12
Name: AMJIND_new, Length: 199523, dtype

,AAGE,ACLSWKR,ADTIND,ADTOCC,AHGA,AHRSPAY,AHSCOL,AMARITL,AMJIND,AMJOCC,...,458,459,460,461,462,463,464,465,466,467
0,73,Not in universe,0,0,High school graduate,0,Not in universe,Widowed,Not in universe or children,Not in universe,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,9,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,10,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199418,9,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
199419,12,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
199420,7,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
199421,60,Self-employed-not incorporated,46,45,High school graduate,0,Not in universe,Married-civilian spouse present,Forestry and fisheries,Farming forestry and fishing,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


Would be good to write code that would loop through all the columns and print out the uniques to add decisions about one-hot vs label encoding vs scaling:

In [ ]:
print(df['ACLSWKR'].unique())
print(df['ACLSWKR'].value_counts())

So that takes one column and generates 9 columns

In [ ]:
from sklearn.preprocessing import LabelEncoder


cols_to_label_encode = ['workex', 'status', 'hsc_s', 'degree_t']
df[cols_to_label_encode] = df[cols_to_label_encode].apply(LabelEncoder().fit_transform)

le = LabelEncoder()
le.fit(["paris", "paris", "tokyo", "amsterdam"])

